### 복습 
- 가전 폴더 안에 모든 데이터파일을 로드해서 하나의 데이터프레임으로 생성 
- 감정에 대한 데이터들이 3개 분류 -> 2개의 분류로 변경 (부정, 중립 -> 부정)
- 감정 데이터가 없는 데이터들은 따로 저장 
- train, test의 비율은 8:2
- Dataset을 기존의 Dataset 구성과 같이 작업
- RawText 데이터를 이용하여 감정분석 모델을 생성 
- SBERT 모델을 이용하여 임베딩 
- 다중퍼셉트론의 모델을 이용하여 감정 분석 (Linear -> ReLU -> DropOut -> Linear)
- 검증 데이터를 이용하여 정확도와 f1_score 확인 
- 감정 데이터가 없는 RawText에서 sample(10)를 출력하여 감정 예측

- 다중퍼셉트론 모델이 아닌 머신러닝 모델 (SVC)을 이용하여 감정 분석 예측

In [ ]:
import os 
import re
import pandas as pd 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sentence_transformers import SentenceTransformer

In [38]:
# 문자 정규화 함수 정의 
def normailize(text):
    text = re.sub(r"[^가-힣0-9a-zA-Z\s\.]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

문자를 정규화하는 이유?
</br>$\Rightarrow$ 데이터에 일관성 부여
</br> <span style="color:#808080"> ***예:** 상품a, 상품 A, 상.품.A $\rightarrow$ 상품A 로 통일* </span>
- 자연어 처리에서 굉장히 중요한 부분
- split(), 외부 라이브러리 OKT, Komoran, Konlpy, KKma : 국어사전을 기준으로 만들어짐
- transformer 기반(Bert 등) : 토큰화 작업을 위한 형태소 분석기를 따로 로드하지 않고 sentencepiece 기반으로 문장을 학습시켜 새로 사전 생성해 토큰화
    - 장점: 국어사전 데이터에 비해 신조어 학습 및 토큰화하기 편함

In [39]:
# 파일의 목록을 로드 -> 목록을 기준으로 데이터를 로드 -> 단순 결합 
file_path = "../data/가전/"
file_list = os.listdir(file_path)
file_list

['3-1.영상음향가전(76).json',
 '3-1.영상음향가전(77).json',
 '3-1.영상음향가전(78).json',
 '3-1.영상음향가전(79).json',
 '3-1.영상음향가전(80).json',
 '3-1.영상음향가전(81).json',
 '3-1.영상음향가전(82).json',
 '3-1.영상음향가전(83).json',
 '3-1.영상음향가전(84).json',
 '3-1.영상음향가전(85).json',
 '3-1.영상음향가전(86).json',
 '3-1.영상음향가전(87).json',
 '3-1.영상음향가전(88).json',
 '3-2.생활미용욕실가전(128).json',
 '3-2.생활미용욕실가전(129).json',
 '3-2.생활미용욕실가전(130).json',
 '3-2.생활미용욕실가전(131).json',
 '3-2.생활미용욕실가전(132).json',
 '3-2.생활미용욕실가전(133).json',
 '3-2.생활미용욕실가전(134).json',
 '3-2.생활미용욕실가전(135).json',
 '3-2.생활미용욕실가전(136).json',
 '3-2.생활미용욕실가전(137).json',
 '3-2.생활미용욕실가전(138).json',
 '3-2.생활미용욕실가전(139).json',
 '3-2.생활미용욕실가전(140).json',
 '3-3.주방가전(127).json',
 '3-3.주방가전(128).json',
 '3-3.주방가전(129).json',
 '3-3.주방가전(130).json',
 '3-3.주방가전(131).json',
 '3-3.주방가전(132).json',
 '3-3.주방가전(133).json',
 '3-3.주방가전(134).json',
 '3-3.주방가전(135).json',
 '3-3.주방가전(136).json',
 '3-3.주방가전(137).json',
 '3-3.주방가전(138).json',
 '3-3.주방가전(139).json',
 '3-4.계절가전(126).json',
 '3-4.계절가전(127)

In [40]:
# 로드한 데이터프레임을 누적으로 결합하기 위해 빈 데이터프레임 생성 
df = pd.DataFrame()

for file in file_list:
    # file : 파일명 
    data = pd.read_json(file_path + file)
    # df에 단순 행 결합 -> df에 다시 대입 
    df = pd.concat( [df, data], axis= 0  )
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4056 entries, 0 to 99
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Index            4056 non-null   int64  
 1   RawText          4056 non-null   object 
 2   Source           4056 non-null   object 
 3   Domain           4056 non-null   object 
 4   MainCategory     4056 non-null   object 
 5   ProductName      4056 non-null   object 
 6   ReviewScore      4056 non-null   int64  
 7   Syllable         4056 non-null   int64  
 8   Word             4056 non-null   int64  
 9   RDate            4056 non-null   int64  
 10  GeneralPolarity  3678 non-null   float64
 11  Aspects          4056 non-null   object 
dtypes: float64(1), int64(5), object(6)
memory usage: 411.9+ KB


In [41]:
# 필요한 컬럼을 제외하고 나머지 컬럼을 무시 
df = df[['RawText', 'GeneralPolarity']]

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4056 entries, 0 to 99
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RawText          4056 non-null   object 
 1   GeneralPolarity  3678 non-null   float64
dtypes: float64(1), object(1)
memory usage: 95.1+ KB


In [43]:
df.rename(columns = {
    'GeneralPolarity' : 'label'
}, inplace=True)

In [44]:
# RawText의 문자 정규화 사용
df['RawText'] = df['RawText'].map(normailize)

In [45]:
# RawText에서 길이가 1 이하인 데이터를 제외
df = df.loc[df['RawText'].str.len() > 1]

In [46]:
# 혹시 RawText에 중복 데이터가 존재할수 있으니 중복 제거 
df.drop_duplicates(subset='RawText', inplace=True)

중복 데이터 제거한 이유: 과적합 방지

In [47]:
# label이 결측치인 데이터는 따로 저장 
na_df = df.loc[df['label'].isna(), ]
na_df

,RawText,label
13,귀에서 자꾸 빠져요.귀에 꼽는재 질이 미끄러운 재질이라 작은 소품이지만 재질만 바꾸...,NaN
43,아이를 출산한 기념으로 TV를 바꿨습니다. 그전에 쓰던 TV가 꽤나 무거워서 떨어지...,NaN
44,화면에 노이즈가 생깁니다. 저희 가족 중에 아무도 TV 화면을 건드리거나 하지 않았...,NaN
55,이번에 이사하면서 우리 따님께서 방에 TV가 있으면 좋겠다고 하여 방에서 사용할 T...,NaN
93,기존에 사용하던 무선이어폰이 오래되어서 배터리가 광탈하는 바람에 새로운 상품이 필요...,NaN
...,...,...
41,대용량이고 세척이 간편하다는 얘기에 구매를 했는데 저는 별로인 거 같아요...생각했...,NaN
44,요거 진짜 진짜 물건입니다. 처음엔 디자인이 너무 귀여워서 주문하게 되었는데요. 작...,NaN
68,지인의 추천으로 믿고 바로 구매를 해서 현재도 사용중입니다좀 더 많은 분들에게 도움...,NaN
76,다른 에어쿨러와 다르게 슬림한 디자인이 마음에 들어요.심플한 디자인 덕분에 집안 어...,NaN


In [48]:
# df에는 결측치를 제외
df = df.loc[ ~df['label'].isna(), ]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3678 entries, 0 to 99
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   RawText  3678 non-null   object 
 1   label    3678 non-null   float64
dtypes: float64(1), object(1)
memory usage: 86.2+ KB


In [49]:
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3678 entries, 0 to 3677
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   RawText  3678 non-null   object 
 1   label    3678 non-null   float64
dtypes: float64(1), object(1)
memory usage: 57.6+ KB


In [50]:
# label의 데이터의 빈도수를 확인 
df['label'].value_counts()

label
 1.0    2220
 0.0     944
-1.0     514
Name: count, dtype: int64

In [51]:
# label의 데이터들을 int형태로 변경 
df['label'] = df['label'].astype(int)

In [52]:
df['label'].value_counts()

label
 1    2220
 0     944
-1     514
Name: count, dtype: int64

In [53]:
# 삼중 분류의 class를 이진 분류로 변경하기 위해 -1을 0으로 변경
# df['label'] = df['label'].map(
#     {
#         -1 : 0, 
#         0 : 0, 
#         1 : 1
#     }
# )

In [54]:
# 삼중 분류의 class를 이진 분류로 변경하기 위해 -1을 0으로 변경
# 삼중 분류하려면 각각에 +1 (0, 1, 2로 변경)
df['label'] = df['label'].map(
    {
        -1 : 0, 
        0 : 1, 
        1 : 2
    }
)

In [55]:
df['label'].value_counts()

label
2    2220
1     944
0     514
Name: count, dtype: int64

In [56]:
# train, test 로 분할
train_df, test_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label']
)

In [57]:
train_df['label'].value_counts()

label
2    1776
1     755
0     411
Name: count, dtype: int64

In [58]:
model_name = 'BM-K/KoSimCSE-roberta-multitask'
sbert = SentenceTransformer(model_name)

No sentence-transformers model found with name BM-K/KoSimCSE-roberta-multitask. Creating a new one with mean pooling.


In [59]:
# 최대 토큰 길이 제한 (너무 길면 오래 걸리니까)
sbert.max_seq_length = 128

In [60]:
# 임베딩, 벡터화

class SBERTHead(Dataset):
    def __init__(self, texts, labels):
        with torch.inference_mode():
            # encode() 함수는 아래의 과정을 한 번에 수행해준다.
            # 문장을 토큰화(만든 단어사전 기반) - 인코딩을 통해 단어 사전에서의 위치값으로 변환 - 벡터화 - 평균값 내기 - normalize embedding이 True라면 정규화 - 텐서화
            #   sentence - tokenizer(vocab) - encoding(vocab index) - meanpool - normalize - tensor
            # tensor화, array화
                # 차이점:
                #   - tensor: pytorch -> deep (다중 퍼셉트론)
                #   - array: sklearn (SVC 모델)
                # 공통점: 행렬
            self.emb = sbert.encode(
                texts, convert_to_tensor=True, normalize_embeddings=True
            )
            self.labels = torch.tensor(labels, dtype=torch.long)
        self.texts = texts
        self.labels2 = labels
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.emb[idx], self.labels[idx]
        # getitem에서 임베딩 처리 
        # res_emb = sbert.encode(
        #     self.texts[idx], convert_to_tensor=True, normalize_embeddings=True
        # )
        # res_label = torch.tensor(self.labels2[idx], dtype=torch.long)
        # return res_emb, res_label

In [62]:
train_ds = SBERTHead(train_df['RawText'].tolist(), train_df['label'].tolist())
test_ds = SBERTHead(test_df['RawText'].tolist(), test_df['label'].tolist())

train_dl = DataLoader(train_ds, batch_size=128, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=256)

In [ ]:
# 다중 퍼셉트론 구조의 분류 모델 생성
# 이진 분류일 때
# class MLPHead(nn.Module):
#     def __init__(self, input_dim, hidden = 256, num_classes = 2, dropout=0.2):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(input_dim, hidden),
#             nn.ReLU(), 
#             nn.Dropout(dropout), 
#             nn.Linear(hidden, num_classes)
#         )
#     def forward(self, x):
#         result = self.net(x)
#         return result

In [70]:
# 다중 퍼셉트론 구조의 분류 모델 생성
# 삼중 분류일 때
# num_classes의 기본값(2) -> 3으로 변경
class MLPHead(nn.Module):
    def __init__(self, input_dim, hidden = 256, num_classes= 2, dropout=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(), 
            nn.Dropout(dropout), 
            nn.Linear(hidden, num_classes)
        )
    def forward(self, x):
        result = self.net(x)
        return result

In [79]:
# sbert 모델에서 출력 차원의 개수를 확인
in_dim = sbert.get_sentence_embedding_dimension()
clf = MLPHead(in_dim, num_classes=3)
# 손실 함수
# 데이터의 불균형 문제를 해결하기 위해, weight 매개변수를 통해 각 class에 가중치 변화를 부여한다.
# 현재 인덱스: [0, 1, 2] -> 적은 데이터일수록 높은 변화
    # 5.0 ~ 10.0 : 비율이 굉장히 적은 데이터
    # 2.0 ~ 4.0 : 비율이 적은 데이터
    # 1.0 : 비율이 가장 많은 데이터
crit = nn.CrossEntropyLoss( weight= torch.tensor([5.0, 3.0, 1.0]) )
# AdamW -> Adam 개선판
opt = optim.AdamW(clf.parameters(), lr=2e-4)

In [80]:
clf.train()

for epoch in range(5):
    total = 0.0
    for x, y in train_dl:
        opt.zero_grad()
        logits = clf(x)
        loss = crit(logits, y)
        loss.backward()
        opt.step()
        total += loss.item() * x.size(0)
    print(f"epoch : {epoch}, loss : {round(total/len(train_df), 4)}")

epoch : 0, loss : 1.0851
epoch : 1, loss : 1.0476
epoch : 2, loss : 0.9954
epoch : 3, loss : 0.9343
epoch : 4, loss : 0.8737


In [81]:
# 정확도, f1-score 확인 
clf.eval()

y_true , y_pred = [], []

with torch.inference_mode():
    for x, y in test_dl:
        logits = clf(x)
        pred = logits.argmax(dim = 1).tolist()
        y_true.extend(y.tolist())
        y_pred += pred


In [82]:
# 이진 분류일 때
# print('accuracy_score', accuracy_score(y_true, y_pred))
# print('f1_score', f1_score(y_true, y_pred))

In [83]:
# 삼중 분류일 때
print('accuracy_score', accuracy_score(y_true, y_pred))
print('f1_score', f1_score(y_true, y_pred, average= 'macro'))

accuracy_score 0.6073369565217391
f1_score 0.625011419791537


In [76]:
samples = na_df['RawText'].sample(10).tolist()
samples

['결혼하고 20년만에 가스레인지를 바꿨습니다. 기능이 너무 많은 것도 화구가 많은 것도 오히려 과유불급이라 심플하면서도 사용이 편리하고 단정한 디자인에 화구의 간격을 생각하면서 제품을 골랐는데 가스레인지 3구는 딱 제가 찾던 심플한 구성의 제품이었습니다. 거기다가 자동으로 불꽃이 조절되고 타이머 기능도 있어 음식을 태우거나 시간을 미처 생각 못해 과 요리하는 일이 없어 정말 편리합니다. 또 건전지 교체 케이스가 전면에 있어 교체하기가 편하고 교체 시기를 알려준다고 하니 좋아요.',
 '매장 직원분이 똑똑 하니 내부가 짠 와 꼭 필요한 기능일까 했는데 용량도 기능도 거거익선 다다익선 내부가 깊고 넓고요 깊어서 손이 안 살짝 닿는 저 같은 사람을 위한 배려 한가지 슬라이딩 선반에요 디테일 하죠 위아래로 높낮이 조절 가능한 무빙선반도 빠지면 섭하죠 쓰는 사람 입장에서 많이 생각한 모양에요 또한 활용도가 높은 수납이 구석구석 있어요멀티수납 2개 있는데 계란 한판 모두 들어가요 깨짐 방지를 위해 충격흡수패드 깔려 있구요 치즈 초컬렛 영양제 파우치 등등 도어 포켓에 보관하면지저분한데 한곳에 모으니 찾기도 쉽고 보기도 좋아요',
 '친정집 보내려 산건데 배송 일주일 넘게 걸렸어요. 배송 느린거 빼면 세탁기는 좋다시네요. 기존에 쓰단 통돌이가 고장나서 새로 구입했는데 액체 세제 넣는 부분이 부실해요. 생각보다 모터힘이 약하니 감안하고 사시길요. 요즘에는 통돌이도 액체 세제 많이 쓰는데 액체 세제 넣는 부분이 너무 작아서 넣을 때마다 고생해서 그냥 옷에 붓는다고 하시네요. 그리고 등급 확인을 못했었네요 3등급인게 마음에 걸리지만 만족합니다. 동작 중에 뚜껑을 열면 동작이 멈추고 뚜껑이 접히는 식이 아니라 윗 선반에 걸려서 활짝 못 여는 게 불편해요 그리고 희안하게 10년 동안 썼던 세탁기와 세탁 마침 멜로디가 같아서 좀 식상하네요.',
 '닭가슴살를 자주 먹는데 덩어리가 너무 커서 소분해 해먹으려고 구입했어요 가 제일 진공 압력이 쎄다고 해서 샀는데 역시 좋네요 먹거리들 특

In [77]:
@torch.no_grad()
def predict_review(texts, batch_size = 128):
    if isinstance(texts, str):
        texts = [texts]
    texts_norm = [normailize(t) for t in texts]  

    sbert.eval()
    clf.eval()

    result = []

    for idx in range(0, len(texts), batch_size):
        batch_text = texts_norm[idx : idx + batch_size]
        embs = sbert.encode(
            batch_text, convert_to_tensor = True, 
            normalize_embeddings = True
        )

        logits = clf(embs)
        probs = logits.softmax(dim = -1)
        preds = probs.argmax(dim = -1).tolist()
        # 이진 분류일 때
        # id2label = {
        #     0 : '부정', 
        #     1 : '긍정'
        # }
        # 삼중 분류일 때
        id2label = {
            0 : '부정', 
            1 : '중립',
            2 : '긍정'
        }
        for idx2, pred in enumerate(preds):
            prob = float(probs[idx2, pred])
            review = texts[idx + idx2]
            label = id2label[pred]
            result.append(
                {
                    'text' : review, 
                    'prob' : prob, 
                    'label' : label
                }
            )
    return result

In [78]:
predict_review(samples)

[{'text': '결혼하고 20년만에 가스레인지를 바꿨습니다. 기능이 너무 많은 것도 화구가 많은 것도 오히려 과유불급이라 심플하면서도 사용이 편리하고 단정한 디자인에 화구의 간격을 생각하면서 제품을 골랐는데 가스레인지 3구는 딱 제가 찾던 심플한 구성의 제품이었습니다. 거기다가 자동으로 불꽃이 조절되고 타이머 기능도 있어 음식을 태우거나 시간을 미처 생각 못해 과 요리하는 일이 없어 정말 편리합니다. 또 건전지 교체 케이스가 전면에 있어 교체하기가 편하고 교체 시기를 알려준다고 하니 좋아요.',
  'prob': 0.665367066860199,
  'label': '긍정'},
 {'text': '매장 직원분이 똑똑 하니 내부가 짠 와 꼭 필요한 기능일까 했는데 용량도 기능도 거거익선 다다익선 내부가 깊고 넓고요 깊어서 손이 안 살짝 닿는 저 같은 사람을 위한 배려 한가지 슬라이딩 선반에요 디테일 하죠 위아래로 높낮이 조절 가능한 무빙선반도 빠지면 섭하죠 쓰는 사람 입장에서 많이 생각한 모양에요 또한 활용도가 높은 수납이 구석구석 있어요멀티수납 2개 있는데 계란 한판 모두 들어가요 깨짐 방지를 위해 충격흡수패드 깔려 있구요 치즈 초컬렛 영양제 파우치 등등 도어 포켓에 보관하면지저분한데 한곳에 모으니 찾기도 쉽고 보기도 좋아요',
  'prob': 0.6068854928016663,
  'label': '긍정'},
 {'text': '친정집 보내려 산건데 배송 일주일 넘게 걸렸어요. 배송 느린거 빼면 세탁기는 좋다시네요. 기존에 쓰단 통돌이가 고장나서 새로 구입했는데 액체 세제 넣는 부분이 부실해요. 생각보다 모터힘이 약하니 감안하고 사시길요. 요즘에는 통돌이도 액체 세제 많이 쓰는데 액체 세제 넣는 부분이 너무 작아서 넣을 때마다 고생해서 그냥 옷에 붓는다고 하시네요. 그리고 등급 확인을 못했었네요 3등급인게 마음에 걸리지만 만족합니다. 동작 중에 뚜껑을 열면 동작이 멈추고 뚜껑이 접히는 식이 아니라 윗 선반에 걸려서 활짝 못 여는 게 불편해요 그리고 희안

**모델 학습이 끝나고, 예측을 통해 성능을 확인하니 만족스럽지 않다. 성능을 올리기 위해 시도할 수 있는 방법은?**
- 데이터를 증가시킨다.
- epoch 수를 증가시킨다. (과적합 위험)
- dropout을 통해 변화를 준다.
- 불균형하게 가중치를 변화한다.